## Word embedding using Glove and similarity metrics (softcosine)

In [1]:
import json
import logging
from re import sub
from multiprocessing import cpu_count

import numpy as np
import pandas as pd 

import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import MatrixSimilarity, SoftCosineSimilarity

import nltk
from nltk.corpus import wordnet as wn

In [2]:
# Reference from
# https://towardsdatascience.com/how-to-rank-text-content-by-semantic-similarity-4d2419a84c32
# https://github.com/4OH4/doc-similarity/blob/master/examples.ipynb
# https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb


In [3]:
# Initialize logging.
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)  # DEBUG # INFO

### Download and/or load the GloVe word vector embeddings and set similarity index

In [4]:
%%time

# Download and/or load the GloVe word vector embeddings
if 'glove' not in locals():  # only load if not already in memory
    glove = api.load("glove-wiki-gigaword-50")
    
similarity_index = WordEmbeddingSimilarityIndex(glove)

Wall time: 18.9 s


### Preprocess Data

In [5]:
# Support functions for pre-processing and calculation

# Import and download stopwords from NLTK.
# nltk.download('stopwords')  # Download stopwords list.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb

# Remove stop words and punctuations with nltk
stopwords = set(nltk.corpus.stopwords.words("english"))
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]
stopwords1 = stopwords.union(punc)

# Remove punctuations and stopwords with sklearn
#from sklearn.feature_extraction import text
#stop_words = text.ENGLISH_STOP_WORDS.union(punc)
def preprocess(doc):
    #return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords1]


# For POS Tagging
def find_pos(new_words,lu_word):
    pt = nltk.pos_tag(new_words)
    pos1 = "None"
    for i in pt:
        if i[0]== lu_word:
            pos1 = (i[1])
        else:
            continue
    return pos1


def add_pos(pos_value):
    # make a dictionary of possible tags (in progress)
    pos_tag = {'verb':['VB','VBD','VBG','VBN','VBP','VBZ'], 
               'noun': ['NN','NNS','NNP'], 
               'adjective': ['JJ','JJR','JJS'],
               'adverb':['RB', 'RBR', 'RBS'],
               'None':['None',''] }

    for ele in pos_tag:
        for i in pos_tag[ele]:
            if i == pos:
                return ele
            else:
                continue

### Parse data from json dictionaries

In [6]:
def extract_ids(dicdata):
    ids = []
    for i in dict_data:
        mea_id = i['id']
        ids.append(mea_id)
    return ids

def extract_meanings(dicdata):
    meaning_list = []
    for i in dict_data:
        meaning = i['meaning'] 
        meaning_list.append(meaning)
    return meaning_list

def extract_pos_meanings(dicdata):
    pos_meaning = []
    for i in dict_data:
        pos = i['fl']
        meaning = i['meaning'] 
        pos_mea = pos + ' '+ meaning # POS + meaning
        pos_meaning.append(pos_mea)
    return pos_meaning

def extract_isteacher(dicdata):
    isteacher = []
    for i in dict_data:
        is_tea = i['isTeacher']
        isteacher.append(is_tea)
    return isteacher

### Softcosine SImilarity
Soft cosine similarity is similar to cosine similarity but in addition considers the semantic relationship between the words through its vector representation.
To compute soft cosines, you will need a word embedding model like Word2Vec or FastText. First, compute the similarity_matrix. Then convert the input sentences to bag-of-words corpus and pass them to the softcossim() along with the similarity matrix.

https://www.machinelearningplus.com/nlp/gensim-tutorial/

250/251 objectionable
229/230 scarcely

In [7]:
# Load Data and store
jsonfile= open('data_600_emb.json','r',encoding='utf8') # removed word aimless
data= jsonfile.read() 

# Parse and Extract word, paragraph, pos and meanings
obj = json.loads(data)

data = obj['data'] #[:10]
print("Total words: ",len(data)) # Total # of words
count=0
# Store accuracy and MRR in blank lists
acc_score=[]
mrr_score=[]

# Parse data and find best meaning
for each_word_data in data:
    word = each_word_data['word']
    paragraph = each_word_data['paragraph']
    dict_data = each_word_data['dictionaryData']
    for i in dict_data:
        ids = extract_ids(dict_data)
        meanings = extract_meanings(dict_data)
        pos_meanings = extract_pos_meanings(dict_data)
        isteacher = extract_isteacher(dict_data)
        
        
    # Preprocess the documents, including the query string(paragraph)
    corpus = []
    for i in pos_meanings:
        corpus.append(preprocess(i))
    query = preprocess(paragraph)
    pos = find_pos(query,word)
    pos_to_list = [add_pos(pos)]
    query_pos = pos_to_list+query



    # Build the model
    # The word embedding model is a large file, so loading is quite a long-running task.

    # Build the term dictionary, TF-idf model
    # The search query must be in the dictionary as well, in case the terms do not overlap with the documents (we still want similarity)
    dictionary = Dictionary(corpus+[query_pos])
    tfidf = TfidfModel(dictionary=dictionary)

    # Create the term similarity matrix. 
    # The nonzero_limit enforces sparsity by limiting the number of non-zero terms in each column. 
    # removing the default value of 100
    similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf, nonzero_limit=None)  # , nonzero_limit=None)


    # Compute Soft Cosine Measure between the query and the documents.
    query_tf = tfidf[dictionary.doc2bow(query)]

    index = SoftCosineSimilarity(
                tfidf[[dictionary.doc2bow(document) for document in corpus]],
                similarity_matrix)

    doc_similarity_scores = (index[query_tf]).round(3)

    # Output the document similarity results
    #sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
    #print(sorted_indexes)

    zipped = list(zip(ids, meanings, doc_similarity_scores, isteacher))
    features = sorted(zipped, key = lambda x: x[2], reverse = True)

    print("\n\nLookup word: ",word)
    ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
    ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
    ranked_meanings['pred_rank'] = ranked_meanings['score'].rank(pct=False, ascending= False)
    print(ranked_meanings)
    
    
    
    count +=1
    print(count)
    
    # Find accuracy
    if ranked_meanings.iloc[0]['isTeacher'] ==1:
        acc_score.append(1)
    else:
        acc_score.append(0)
        
    # Find MRR score
    istea_index = ranked_meanings[ranked_meanings['isTeacher']==1].index[0]+1
    pred_rank =1/istea_index
    mrr_score.append(pred_rank)

# Return Accuracy and MRR 
#print(acc_score)
avg = sum(acc_score)/len(acc_score)
print("The accuracy score is ", round(avg,4))

mrr = np.sum(mrr_score)/ len(mrr_score)
print("The Mean Reciprocal Rank is ", round(mrr,4)) 

Total words:  593


Lookup word:  circumvent 
     id                                            meaning  score  isTeacher  \
0  3230  to avoid being stopped by (something, such as ...  0.309          1   

   pred_rank  
0        1.0  
1


Lookup word:  addict
     id                                            meaning  score  isTeacher  \
0  3401  a person who is not able to stop taking drugs ...  0.403        0.0   
1  5885                    to cause addiction in (someone)  0.371        0.0   
2  5884  one exhibiting a compulsive, chronic, physiolo...  0.339        1.0   

   pred_rank  
0        1.0  
1        2.0  
2        3.0  
2


Lookup word:  frustrated
     id                                            meaning  score  isTeacher  \
0  5886  feeling, showing, or characterized by frustrat...  0.152        1.0   
1  5887                   to balk or defeat in an endeavor  0.035        0.0   

   pred_rank  
0        1.0  
1        2.0  
3


Lookup word:  manslaughter
     id    



Lookup word:  zeal
     id                                            meaning  score  isTeacher  \
0  3319  a strong feeling of interest and enthusiasm th...  0.429        1.0   
1  5935                                   great enthusiasm  0.324        0.0   
2  5934  eagerness and ardent interest in pursuit of so...  0.219        0.0   
3  3320                     of or relating to missionaries  0.038        0.0   

   pred_rank  
0        1.0  
1        2.0  
2        3.0  
3        4.0  
22


Lookup word:  convictions
     id                                            meaning  score  isTeacher  \
0  2576  the ability to do something that you know is d...  0.560        0.0   
1  5937       the courage to do what one believes is right  0.433        1.0   
2  5936  the act or process of finding a person guilty ...  0.221        0.0   
3  2575  the act of proving that a person is guilty of ...  0.161        0.0   

   pred_rank  
0        1.0  
1        2.0  
2        3.0  
3        4.

C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))




Lookup word:  befuddled
     id                                         meaning  score  isTeacher  \
0  2502         unable to think clearly : very confused  0.157        0.0   
1  5945  utterly confused or puzzled : deeply perplexed  0.122        1.0   
2  5946   to muddle or stupefy with or as if with drink  0.000        0.0   

   pred_rank  
0        1.0  
1        2.0  
2        3.0  
26


Lookup word:  damning
     id                                            meaning  score  isTeacher  \
0  5947                                 bringing damnation  0.363        0.0   
1  3476  —used to show that you are angry or annoyed at...  0.296        0.0   
2  5949  causing or leading to one's own condemnation o...  0.280        1.0   
3  5948  to condemn to a punishment or fate; especially...  0.226        0.0   
4  3477                                 yourself or itself  0.034        0.0   

   pred_rank  
0        1.0  
1        2.0  
2        3.0  
3        4.0  
4        5.0  
27


Lo

C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))


...



Lookup word:  Mortifying
     id                                            meaning  score  isTeacher  \
0  6978  to subject to severe and vexing embarrassment ...    0.0        1.0   
1  6977  causing feelings of strong shame or embarrassment    0.0        0.0   

   pred_rank  
0        1.5  
1        1.5  
578


Lookup word:  impressionable
     id                            meaning  score  isTeacher  pred_rank
0  6982  capable of being easily impressed  0.201          1        1.0
579


Lookup word:  tabula
     id                                            meaning  score  isTeacher  \
0  6983  the mind in its hypothetical primary blank or ...   0.21          1   

   pred_rank  
0        1.0  
580


Lookup word:  wrench
     id                                            meaning  score  isTeacher  \
0  6990  to damage or change (something) in a way that ...  0.781        0.0   
1  6991  to damage or change (something) in a way that ...  0.781        0.0   
2  6989  a wrench usual

C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))




Lookup word:  habitation
     id                            meaning  score  isTeacher  pred_rank
0  6995  the act of inhabiting : occupancy    0.0          1        1.0
583


Lookup word:  patronizing
     id                                            meaning  score  isTeacher  \
0  2700  to give money or support to (someone or someth...  0.169        0.0   
1  2699  showing that you believe you are more intellig...  0.152        1.0   

   pred_rank  
0        1.0  
1        2.0  
584


Lookup word:  converging
     id                                            meaning  score  isTeacher  \
0  6997  to tend or move toward one point or one anothe...  0.532        1.0   
1  6998  a lens that causes parallel rays (as of light)...  0.184        0.0   

   pred_rank  
0        1.0  
1        2.0  
585


Lookup word:  diverging
     id                                            meaning  score  isTeacher  \
0  6999  to move or extend in different directions from...  0.306          1   

   

C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))




Lookup word:  fiddles 
     id                                            meaning  score  isTeacher  \
0  7013  to move or handle (something) with the hands o...  0.741        1.0   
1  7010                                    to fritter away  0.508        0.0   
2  7012  to spend time in activity that does not have a...  0.297        0.0   
3  7011    one that plays a supporting or subservient role  0.189        0.0   
4  7007                                to play on a fiddle  0.092        0.0   
5  7008                                        double bass  0.072        0.0   
6  7009                                        double bass  0.072        0.0   
7  7006                                             violin  0.000        0.0   

   pred_rank  
0        1.0  
1        2.0  
2        3.0  
3        4.0  
4        5.0  
5        6.5  
6        6.5  
7        8.0  
592


Lookup word:  indebted
     id                                            meaning  score  isTeacher  \
0  7014  o